<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---


<p align="center"><h1 align="center"> Image Classification Model Submission Guide - Predictions Only (No Model Metadata Extraction)

---
Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data
3. Fit model on preprocessed data
4. Generate predictions from X_test data and submit to competition
5. Repeat submission process to improve place on leaderboard



## 1. Get data in and set up X_train, X_test, y_train objects

In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [2]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/neuralnet_subclasses_competition_data-repository:latest') 


Data downloaded successfully.


In [ ]:
# Create training data objects 

# Extract images
!unzip "neuralnet_subclasses_competition_data/X_train.zip"
!unzip "neuralnet_subclasses_competition_data/X_test.zip" 

# Create ordered list of filepaths 
train_filepaths = [('/content/train_shuffle/' + str(i) + '.jpg') for i in range(0, 6472)]
test_filepaths = [('/content/test_shuffle/' + str(i) + '.jpg') for i in range(0, 9127)]

# Read in y_train data 
import pandas as pd 
y_train = pd.read_csv("neuralnet_subclasses_competition_data/y_train.csv")

##2.   Preprocess data


In [ ]:
# Write and execute code to preprocess data here - an example is provided below 

In [4]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently

def preprocessor(image_filepath, shape=(10, 10)):
        """
        This function preprocesses reads in images, resizes them to a fixed shape and
        min/max transforms them before converting feature values to float32 numeric values
        required by onnx files.
        
        params:
            image_filepath
                full filepath of a particular image
                      
        returns:
            X
                numpy array of preprocessed image data
                  
        """
           
        import cv2
        import numpy as np

        "Resize a color image and min/max transform the image"
        img = cv2.imread(image_filepath) # Read in image from filepath.
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 reads in images in order of blue green and red, we reverse the order for ML.
        img = cv2.resize(img, shape) # Change height and width of image.
        img = img / 255.0 # Min-max transform.


        # Resize all the images...
        X = np.array(img)
        X = np.expand_dims(X, axis=0) # Expand dims to add "1" to object shape [1, h, w, channels] for keras model.
        X = np.array(X, dtype=np.float32) # Final shape for onnx runtime.
        return X

In [5]:
# Use preprocessor to create X_train object 

# Import image, load to array of shape height, width, channels, then min/max transform...
# Read in all images from filenames...
import numpy as np 
import os 

preprocessed_image_data = [preprocessor(x) for x in train_filepaths]

# Object needs to be an array rather than a list for Keras. (vstack converts above list to array object.)
X_train = np.vstack(preprocessed_image_data)
# Assigning to X_train to highlight that this represents feature input data for our model.

In [6]:
# Preprocess X_test image data to generate predictions from models 
import numpy as np

preprocessed_image_data = [preprocessor(x) for x in test_filepaths]

#Create single X_test array from preprocessed images
X_test = np.vstack(preprocessed_image_data) 

##3. Fit model on preprocessed data


In [ ]:
# Write and execute code to fit model on preprocessed data here.

In [7]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://g1waf0iu3m.execute-api.us-east-1.amazonaws.com/prod/m"
#This is the unique rest api that powers this specific Image Classification Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [8]:
#Instantiate Competition
import aimodelshare as ai
mycompetition= ai.Competition(apiurl)

## **4. Submit Model predictions to leaderboard (without extracting model architecture information):**
- model metadata extraction allows you use compare_models() and instantiate_model() functions.

In [9]:
#create list of y category labels
y_labels = list(y_train.columns)

In [10]:
#Generate a list of predictions using X_test data

# This example uses randomly chosen values from y_labels to generate a list of predictions
import random 
predicted_values = random.choices(y_labels, k=len(X_test))

In [11]:
#Submit Model predictions to leaderboard (without extracting model architecture information): 

# Submit to Competition Leaderboard
mycompetition.submit_model(model_filepath = None,
                                 preprocessor_filepath=None,
                                 prediction_submission=predicted_values)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 4

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2641


In [12]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,deep_learning,model_type,depth,num_params,dropout_layers,flatten_layers,maxpooling2d_layers,dense_layers,conv2d_layers,relu_act,softmax_act,loss,optimizer,memory_size,username,version
0,0.33%,0.34%,0.24%,0.64%,keras,True,Sequential,5.000000,578265.000000,nan,1.000000,nan,4.000000,nan,3.000000,1.000000,str,RMSprop,2314112.000000,COMS_NNDL,2
1,0.48%,0.32%,0.20%,1.08%,keras,True,Sequential,11.000000,15197.000000,2.000000,1.000000,2.000000,2.000000,4.000000,5.000000,1.000000,str,RMSprop,62480.000000,COMS_NNDL,3
2,0.28%,0.28%,0.22%,0.79%,keras,True,Sequential,11.000000,15197.000000,2.000000,1.000000,2.000000,2.000000,4.000000,5.000000,1.000000,str,RMSprop,62480.000000,COMS_NNDL,1
3,0.24%,0.32%,0.24%,0.48%,unknown,nan,unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,COMS_NNDL,4


You can also compare two or more models for any models submitted to the leaderboard using example code for model metadata extraction (see code tab for this competition at www.modelshare.org for submission examples.)
```
data=mycompetition.compare_models([1,2], verbose=1)
mycompetition.stylize_compare(data)
```



#####  (Optional Extension) Submit Model With Custom Metadata: 
Can use to add team names or any other missing data you may wish to share on the leaderboard


In [ ]:
# Custom metadata can be added by passing a dict to the custom_metadata argument of the submit_model() method
# This option can be used to fill in missing data points or add new columns to the leaderboard

custom_meta = {'team': 'team one',
               'model_type': 'your_model_type',
               'new_column': 'new metadata'}

mycompetition.submit_model(model_filepath = None,
                                 preprocessor_filepath=None,
                                 prediction_submission=predicted_values,
                                 custom_metadata = custom_meta)